# KaKr 3rd Competition - Car Model Classification

baseline kernel (https://www.kaggle.com/fulrose/3rd-ml-month-car-model-classification-baseline)을 참고하여 공부함을 말씀드립니다.

### Data
* train.csv - Train 셋의 이미지 파일명, 바운딩박스 좌표, 차종정보
* test.csv - Test 셋의 이미지 파일명, 바운딩박스 좌표, 차종정보
* submission.csv - Test셋과 대응되는 제출 파일
* class.csv - 데이터 셋의 class 컬럼과 대응되는 차종의 레이블
* train.zip - Train 이미지 파일
* test.zip - Test 이미지 파일

In [38]:
# Load Libraries
import os
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import glob
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from keras import backend as K

import warnings
warnings.filterwarnings(action='ignore')

import os
path = '../input'
print(os.listdir(path))

['test', 'class.csv', 'sample_submission.csv', 'test.csv', 'train.csv', 'train']


In [39]:
# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(path, 'train')
TEST_IMG_PATH = os.path.join(path, 'test')

# CSV 파일 경로
df_train = pd.read_csv(os.path.join(path, 'train.csv'))
df_test = pd.read_csv(os.path.join(path, 'test.csv'))
df_class = pd.read_csv(os.path.join(path, 'class.csv'))

## Exploratory Data Analysis
* img_file - 데이터 셋의 각 로우와 연결되는 이미지 파일 이름
* bbox_x1 - 바운딩 박스 x1 좌표 (좌상단 x)
* bbox_y1 - 바운딩 박스 y1 좌표 (좌상단 y)
* bbox_x2 - 바운딩 박스 x2 좌표 (우하단 x)
* bbox_y2 - 바운딩 박스 y2 좌표 (우하단 y)
* class - 예측하려는 차종(Target)
* id - 각 데이터 셋에 기입 되어 있는 클래스 id
* name - 클래스 id에 대응되는 실제 차종 레이블

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_class.head()

In [ ]:
print('Target Class Num : ', format(df_class.shape[0]))
print('Target Class Num of Train Data : ', format(df_train['class'].nunique()))

### Class distribution
DataSet의 Class 분포를 확인하여 밸런스를 체크하고자 합니다.

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(df_train["class"], order=df_train["class"].value_counts(ascending=True).index)
plt.title("Number of data per each class")
plt.show()

특정한 class의 data가 더 많음을 알 수 있습니다.

In [ ]:
cntEachClass = df_train["class"].value_counts(ascending=False)

In [ ]:
print("Most Count Value  : {}".format(cntEachClass.max()))
print("Most Count Class Num : {}".format(cntEachClass.index[0]))
print("Fewest Count Value : {}".format(cntEachClass.min()))
print("Fewest Count Class Num : {}".format(cntEachClass.index[-1]))

print("Mean count : {}".format(cntEachClass.mean()))

* 119번 Class가 84개 있으며, 136번 Class가 30개 존재함과 더불어 모든 Class의 데이터 평균은 51개입니다.

## Image Visualization
PIL 라이브러리를 사용하여, 파이썬 커널에서 이미지를 로드합니다.

In [ ]:
import PIL
from PIL import ImageDraw

tmp_imgs = df_train['img_file'][100:110]
plt.figure(figsize=(12,20))

for num, f_name in enumerate(tmp_imgs):
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
    plt.subplot(5, 2, num + 1)
    plt.title(f_name)
    plt.imshow(img)
    plt.axis('off')

## Bounding box
* 바운딩박스는 이미지 내부에서 특정 Object를 박스로 레이블한 좌표를 말하며, 보통 좌측 상단 (x1, y1)과 우측 하단 (x2, y2) 좌표가 주어져서 직사각형 모양의 박스를 그릴 수 있게 됩니다. 이 때, 좌표는 이미지의 픽셀 좌표입니다.
* 아래에서는 위의 image에 Bounding box를 그려주도록 합니다.

In [ ]:
def draw_rect(drawcontext, pos, outline=None, width=0):
    (x1, y1) = (pos[0], pos[1])
    (x2, y2) = (pos[2], pos[3])
    points = (x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1)
    drawcontext.line(points, fill=outline, width=width)

def make_boxing_img(img_name) :
    if img_name.split('_')[0] == "train" :
        PATH = TRAIN_IMG_PATH
        data = df_train
    elif img_name.split('_')[0] == "test" :
        PATH = TEST_IMG_PATH
        data = df_test
        
    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, \
                   ['bbox_x1','bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)
    draw = ImageDraw.Draw(img)
    draw_rect(draw, pos, outline='red', width=10)
    
    return img

make_boxing_img는 image file명을 입력하면 pos, draw를 저장시킨 이후 draw_rect를 통해 image에 box를 그려줍니다.

In [ ]:
f_name = "train_00102.jpg"

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
# Original Image
origin_img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
plt.title("Original Image - {}".format(f_name))
plt.imshow(origin_img)
plt.axis('off')

# Image included bounding box
plt.subplot(1, 2, 2)
boxing = make_boxing_img(f_name)
plt.title("Boxing Image - {}".format(f_name))
plt.imshow(boxing)
plt.axis('off')

plt.show()

* 왼쪽과 같은 image는 필요로하는 Target object와 상관 없는 다른 Object(noise)가 섞여 있을 수 있기에 이미지 내부에서 필요한 Object를 명확히 하기 위해 Bounding box를 사용합니다. (실제로 이미지를 모델에 넣을 때는 Box 부분만을 Crop하여 사용합니다.

# Image Crop
허태명님의 커널(https://www.kaggle.com/tmheo74/3rd-ml-month-car-image-cropping)을 따라 작성했습니다. 
태명님께서 Crop 이미지 데이터는 Dataset으로 만들어 놓으셨습니다.(https://www.kaggle.com/tmheo74/3rd-ml-month-car-image-cropping-dataset)

In [ ]:
'''
def crop_boxing_img(img_name, margin=16) :
    if img_name.split('_')[0] == "train" :
        PATH = TRAIN_IMG_PATH
        data = df_train
    elif img_name.split('_')[0] == "test" :
        PATH = TEST_IMG_PATH
        data = df_test
        
    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, \
                   ['bbox_x1','bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)

    width, height = img.size
    x1 = max(0, pos[0] - margin)
    y1 = max(0, pos[1] - margin)
    x2 = min(pos[2] + margin, width)
    y2 = min(pos[3] + margin, height)

    return img.crop((x1,y1,x2,y2))
'''

In [ ]:
'''
# Train Image Crop
for i, row in df_train.iterrows():
    cropped = crop_boxing_img(row['img_file'])
    cropped.save(row['img_file'])
    
# Test Image Crop
for i, row in df_test.iterrows():
    cropped = crop_boxing_img(row['img_file'])
    cropped.save(row['img_file'])
'''

In [ ]:
'''
# Crop Image Check
tmp_imgs = df_train['img_file'][100:105]
plt.figure(figsize=(12,20))

for num, f_name in enumerate(tmp_imgs):
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
    plt.subplot(5, 2, 2*num + 1)
    plt.title(f_name)
    plt.imshow(img)
    plt.axis('off')
    
    img_crop = PIL.Image.open(f_name)
    plt.subplot(5, 2, 2*num + 2)
    plt.title(f_name + ' cropped')
    plt.imshow(img_crop)
    plt.axis('off')
'''

In [ ]:
'''
import zipfile

with zipfile.ZipFile('train_crop.zip','w') as zip: 
        # writing each file one by one 
        for file in glob.glob('train*.jpg'): 
            zip.write(file)
            
with zipfile.ZipFile('test_crop.zip','w') as zip: 
        # writing each file one by one 
        for file in glob.glob('test*.jpg'): 
            zip.write(file)
'''

# Modeling

### train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
